In [274]:
import pandas as pd
import numpy as np
import itertools

In [275]:
fingers = ['1', '2', '3', '4', '5']

# Calculating valid sequences with rules

In [276]:
presses_per_finger= {
    1:1,
    2:2,
    3:1,
    4:2,
    5:1,
}

In [277]:
def check_illegal_seq(seq):
    #check runs of 3
    for pos in range(len(seq)- 2):
        if (int(seq[pos]) == int(seq[pos + 1]) + 1 and int(seq[pos+1]) == int(seq[pos+2]) + 1):
            return True
        if (int(seq[pos]) == int(seq[pos + 1]) - 1 and int(seq[pos+1]) == int(seq[pos+2]) - 1):
            return True
    
    #check double press
    
    for pos in range(len(seq) - 1):
        if(int(seq[pos]) == int(seq[pos+1])):
            return True
        

    for finger in fingers:
        if seq.count(finger) == 0:
            return True
        

    # # match presses per finger 
    # for finger in fingers:
    #     if seq.count(finger) != presses_per_finger[int(finger)]:
    #         return True
        
    return False

In [278]:
check_illegal_seq('1243542')

False

In [279]:
valid_sequences = []

# generate all possible sequences of 7 digits from 1 to 5
for seq in itertools.product(range(1,6), repeat=7):
    seq = ''.join(map(str, seq))
    if not check_illegal_seq(seq):
        valid_sequences.append(seq)

In [280]:
len(valid_sequences)

5218

# Generating Target Files

In [282]:
windowSizes = ['1', '2', '3', '7']

iti = 3000 
execTime = 10000 # msecs for each trial maximum
precueTime_interval = [600, 1000] # msecs for planning before movement 
hand = 2 #left or right hand
num_trained_seq = 2

total_sub_num = 20
num_sessions = 3
num_blocks_per_session = 12


In [283]:
def determine_trained_seq_for_subj(subj_num):
    np.random.seed(subj_num)
    # while True:
    #     selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    #     if sum([selected_sequences[0][i] == selected_sequences[1][i] for i in range(7)]) <= 2 and selected_sequences[0][0] != selected_sequences[1][0]:
    #         return selected_sequences

    selected_sequences = np.random.choice(valid_sequences, num_trained_seq, replace=False)
    return selected_sequences


In [284]:
for subnum in range(total_sub_num):
    print(determine_trained_seq_for_subj(subnum))

['2423251' '3415213']
['1214532' '2451532']
['3254124' '3241513']
['1324353' '1413521']
['2125341' '3412151']
['2415313' '1453132']
['5312141' '4152413']
['1325451' '1451352']
['4524213' '4352541']
['4351325' '3542521']
['3512434' '3142451']
['1243545' '2153414']
['3241425' '3531423']
['2515342' '4215315']
['5241535' '5153423']
['2353145' '2524513']
['4532351' '1324534']
['5243135' '5421423']
['3245341' '5154243']
['2142523' '5143542']


In [285]:
trained_symbols = ['#', '$']
random_symbol = '%'

In [286]:
### target file headers: [subNum, hand, isTrain, symbol, seq, isMasked, execTime, iti, precueTime, windowSize]

for sub in range(total_sub_num):
    
    subNum = sub + 1
    hand = 2
    trained_seqs = determine_trained_seq_for_subj(subNum)
    num_rep_rand = 2


    for session in range(num_sessions):
        for blocknum in range(num_blocks_per_session):
            block = pd.DataFrame(columns=["subNum", "hand", "isTrain", "symbol", "seq", "isMasked", "execTime", "iti", "precueTime", "windowSize"])

            # Trained Trials
            for seq_idx, seq in enumerate(trained_seqs):
                symbol = trained_symbols[seq_idx]
                for windowSize in windowSizes:
                    #Memory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, symbol, seq, 1, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)
                    #Memory sensory trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 1, symbol, seq, 0, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)


            # Random Trials
            for windowSize in windowSizes:
                for _ in range(num_rep_rand):
                    while True:
                        seq = np.random.choice(valid_sequences)
                        if seq not in trained_seqs:
                            break
                    symbol = random_symbol
                    # Sensory only trial
                    precueTime = np.random.randint(precueTime_interval[0], precueTime_interval[1])
                    trial = pd.DataFrame([[subNum, hand, 0, symbol, seq, 0, execTime, iti, precueTime, windowSize]], columns = block.columns)
                    block = pd.concat([block, trial], ignore_index=True)

            block = block.sample(frac=1).reset_index(drop = True)
            block.to_csv(f'SMSH1_subj{subNum}_session{session+1}_block{blocknum+1}.tgt', index=False, sep='\t')




    
    
    
    